In [1]:
import numpy as np

from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

from tensorflow.keras.datasets import cifar10


In [2]:
NUM_CLASSES = 10


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [4]:
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, NUM_CLASSES)
y_test  = to_categorical(y_test, NUM_CLASSES)

print(x_train[54, 12, 13, 1])

0.36862746


In [7]:
# architecture
input_layer = Input(shape=(32, 32, 3))

conv_layer_1 = Conv2D(
    filters = 10
    , kernel_size = (4, 4)
    , strides = 2
    , padding = 'same'
)(input_layer)

conv_layer_2 = Conv2D(
    filters = 20
    , kernel_size = (3, 3)
    , strides = 2
    , padding = 'same'
)(conv_layer_1)

flatten_layer = Flatten()(conv_layer_2)

output_layer = Dense(units=10, activation='softmax')(flatten_layer)

model = Model(input_layer, output_layer)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 10)        490       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 20)          1820      
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                12810     
Total params: 15,120
Trainable params: 15,120
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Conv2D + Batch Normalization, Activation (Leaky ReLU), Dropout を使う
# BAD (Batch Normalization, Activation, Dropout) で覚える（Dropout は入れていない部分もある）
# Batch Normalization と Activation の順番は逆でもよい（決まったルールはない）
# 最近では過学習対策で Dropout を使わず Batch Normalization のみで対応することも増えている

input_layer = Input(shape=(32, 32, 3))

x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters=32, kernel_size=3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Flatten()(x)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x)

revised_model = Model(input_layer, output_layer)

In [10]:
revised_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0   

In [11]:
# train
optimizer = Adam(lr = 0.0005)
revised_model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
revised_model.fit(x_train, y_train, batch_size=32, epochs=10, shuffle=True, validation_data=(x_test, y_test))

Epoch 1/10
1563/1563 [==============================] - 56s 36ms/step - loss: 1.5300 - accuracy: 0.4637 - val_loss: 1.4180 - val_accuracy: 0.4975
Epoch 2/10
1563/1563 [==============================] - 71s 45ms/step - loss: 1.1280 - accuracy: 0.6003 - val_loss: 1.1718 - val_accuracy: 0.5976
Epoch 3/10
1563/1563 [==============================] - 81s 52ms/step - loss: 0.9881 - accuracy: 0.6534 - val_loss: 0.9614 - val_accuracy: 0.6614
Epoch 4/10
1563/1563 [==============================] - 78s 50ms/step - loss: 0.9046 - accuracy: 0.6844 - val_loss: 0.8976 - val_accuracy: 0.6867
Epoch 5/10
1563/1563 [==============================] - 82s 53ms/step - loss: 0.8426 - accuracy: 0.7083 - val_loss: 0.9076 - val_accuracy: 0.6837
Epoch 6/10
1563/1563 [==============================] - 81s 52ms/step - loss: 0.7917 - accuracy: 0.7244 - val_loss: 0.8356 - val_accuracy: 0.7163
Epoch 7/10
1563/1563 [==============================] - 84s 54ms/step - loss: 0.7486 - accuracy: 0.7393 - val_loss: 0.8371 -

In [15]:
revised_model.layers

In [17]:
len(revised_model.layers)

20

In [19]:
revised_model.layers[6]

In [21]:
revised_model.layers[5].get_weights()

[array([0.94041705, 0.9054515 , 0.89516234, 0.87409675, 0.90987194,
        0.9342497 , 0.9350598 , 0.9442752 , 1.0684077 , 1.25339   ,
        1.0328705 , 1.0219436 , 1.1270845 , 0.96516013, 1.067706  ,
        0.9814122 , 0.9114092 , 1.0136174 , 0.8089466 , 0.98778194,
        1.2550875 , 1.2923557 , 0.8400846 , 0.9152235 , 0.81022143,
        1.0261496 , 1.1428882 , 0.9416784 , 0.87324536, 0.8812449 ,
        0.9735475 , 0.8953181 ], dtype=float32),
 array([-0.1162336 ,  0.01513166, -0.14074613, -0.17746873, -0.2139352 ,
        -0.14468959,  0.01926365, -0.07801389, -0.14280714, -0.17310649,
        -0.1390724 , -0.23974603, -0.15428822, -0.06365444,  0.00854171,
        -0.13585724, -0.15001576, -0.20856644, -0.18348838, -0.11919554,
        -0.1115865 , -0.101971  , -0.229456  , -0.07921709, -0.17097126,
        -0.0127638 , -0.16182753, -0.06875672, -0.236037  , -0.19673002,
        -0.1561337 , -0.00070475], dtype=float32),
 array([-0.11198465, -2.434334  ,  0.48006782, -0.0204

In [22]:
revised_model.layers[6].get_weights()

[]

In [24]:
revised_model.evaluate(x_test, y_test)

313/313 [==============================] - 4s 12ms/step - loss: 0.9046 - accuracy: 0.7035


[0.9046057462692261, 0.703499972820282]